In [14]:
import copy, math, time, random
random.seed(103)

1. Play 100 games of random player against random player
    - What percentage of games does each player (1st or 2nd) win?
    - On average, how many moves does it take to win?
2. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
3. Play 100 games with the random player against the minimax AI player at a depth of 5 plies
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Is your AI player better than random chance? Write a paragraph or two describing why or why not.
4. Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not.
5. (Extra Credit, 10 points). Play 100 games with the random player against the
    - Alpha-Beta AI player at a depth of 10 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Does increasing the number of plies improve the play for our AI player? Why or why not?

explanation for deepcopy - copy library
https://www.scaler.com/topics/copy-in-python/

In [15]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1
    
        self.num_plays = 0
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        self.p1_wins = 0
        self.p2_wins = 0
        self.ties = 0
        
    def copy(self):
        # Create a new instance of the Mancala class
        new_mancala = Mancala()

        # Copy the relevant state information to the new instance
        new_mancala.board = self.board.copy()  # Assuming board is a list or another mutable object
        new_mancala.current_player = self.current_player
        # Copy any other relevant state information

        return new_mancala
        

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                       self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                      self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        
        # random.seed(69)
        random_move = random.randint(1, self.pits_per_player)
        # print("maybe I'll do this move", random_move)
        while self.valid_move(random_move) == False:
            random_move = random.randint(1, self.pits_per_player)
            # print("maybe I'll do this move", random_move)
        
        return random_move

        # write your code here
        pass
        
    def generate_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player+1) * 2)
        self.current_player = 1
        self.moves = []
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        pit_index = 0
        if self.current_player == 1:
            pit_index = pit - 1
        else:
            pit_index = pit + self.pits_per_player
            
        # make sure pit is within bounds
        if (0 <= pit_index < len(self.board)):
            # make sure pit is not a player's mancala
            if (pit_index != self.pits_per_player) and (pit_index != (self.pits_per_player*2)+1):
                # make sure pit has stones in it
                if self.board[pit_index] != 0:
                    # add the move to the list of valid moves
                    move = self.current_player, pit
                    self.moves.append(move)
                    return True                    
        return False
        
        pass

    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        # 1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        if self.valid_move(pit) == False:
            #print("INVALID MOVE")
            return self.board
        
        
        # # 2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        # if self.terminal_state() == True:
        #     # print("GAME OVER")
        #     return self.board
        
        
        # 3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.
        
        # init variables
        pit_start = 0
        other_player_mancala = 0
        board_size = len(self.board)
        current_index = 0
        empty_pit_stones = 0
        
        
        # update the pit index depending on the player
        # pit index will be used to access the pit of choice and move the stones
        
        if self.current_player == 1:
            pit_start = pit
            other_player_mancala_index = self.pits_per_player*2 + 1
            current_player_mancala_index = self.pits_per_player
        else:
            pit_start = pit + 1 + self.pits_per_player
            other_player_mancala_index = self.pits_per_player
            current_player_mancala_index = self.pits_per_player*2 + 1
            
        
        # grab stones from specified pit
        stones = self.board[pit_start-1]
        self.board[pit_start-1] = 0
        
        
        # distribute stones across the board, skipping over the other player's mancala
        i=0
        
        while i<stones:
            current_index = (i+pit_start)%board_size
            if current_index == other_player_mancala_index:
                stones += 1
            else:
                self.board[current_index] += 1
            i += 1        

        
        # check if the last stone was dropped into an empty pit on players side
        # if so take all stones from this current pit and the opposite pit
        adjacent_index = abs(current_index - (current_index-self.pits_per_player)*2)
        # make sure pit is not a player's mancala
        if (current_index != self.pits_per_player) and (current_index != ((self.pits_per_player*2)+1)):
            if self.board[current_index] == 1:
                # make sure that we are on the current player's side
                
                if self.current_player == 1 & current_index in self.p1_pits_index:
                    self.board[current_player_mancala_index] += self.board[current_index]+self.board[adjacent_index]
                    self.board[current_index] = 0
                    self.board[adjacent_index] = 0
                if self.current_player == 2 and current_index in self.p2_pits_index:
                    self.board[current_player_mancala_index] += self.board[current_index]+self.board[adjacent_index]
                    self.board[current_index] = 0
                    self.board[adjacent_index] = 0
                
        
        # Finally, the function then switches the current player, allowing the other player to take their turn.
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1
        
        return self.board
        
    def terminal_state(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        winning_state = True
        # check player 1's side
        for i in range(0, self.pits_per_player):
            if self.board[i] != 0:
                winning_state = False
        
        if winning_state == True:
            # utility function
            return True
        
        winning_state = True
        
        # check player 2's side
        for i in range(self.pits_per_player+1, self.pits_per_player*2+1):
            if self.board[i] != 0:
                winning_state = False
        if winning_state == True:
            return True
        
        return False

In [16]:
# this will run a certain number of games so we can test an
# AI player which uses the MiniMax algorithm against a raondomly moving player
def random_vs_random(games = 100):

        p1_count = 0
        p2_count = 0
        ties_count = 0
        move_count = 0

        player_turn = 1
        # play 100 games
        for i in range(games):
            game = Mancala()

            # game.play(1)
            while not game.terminal_state():
                # game.display_board()
                # print(game.current_player)
                if game.current_player == 2:
                    # print("The game 'board' says its this players turn:", game.current_player)
                    # print("Random player taking their turn")
                    game.play(game.random_move_generator())
                else:
                    # print("The game 'board' says its this players turn:", game.current_player)
                    # print("AI player is taking their turn, here is the current board")
                    game.play(game.random_move_generator())
                move_count += 1
            # game.display_board()
            if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
                p1_count += 1
                # print("Player 1 won that round")
            elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
                p2_count +=1
                # game.display_board()
                # print("Player 2 won that round")
            else:
                ties_count += 1
                # print("tie on that round")


        print("Player 1 won:", p1_count/games, "% of the time, and Player 2 won:", p2_count/games,"of the time, in an average of:", move_count/games,"moves, ties count", ties_count)

        pass

random_vs_random()

Player 1 won: 0.14 % of the time, and Player 2 won: 0.77 of the time, in an average of: 79.22 moves, ties count 9


In Random vs Random for some reason player 1 wins about 15% of the time and player 2 wins about 75% of the time. This used to output 50/50 but after changing up some code to make minimax run more efficiently (none of which would logically have changed the random function at all...) this function started outputting strainge results. I have debugged this for hours alone and with the help of two different TAs and no one could figure out why this is happening. The random function spits out consistently random moves.

In [5]:
# Here we will implement the AI for mancala using a Minimax algorithem





def possible_valid_moves(state):
        PVM = []
        for j in range(state.pits_per_player+1):
                if state.valid_move(j) == True:
                    PVM.append(j)
        return PVM
    
def evaluate_state(state):
        # Utility function  :- Difference between P1 mancala and p2 mancala
        p1Stones = state.board[(state.pits_per_player)]
        p2Stones = state.board[((state.pits_per_player*2)+1)]
        # return p1Stones - p2Stones
        
        return p1Stones - p2Stones
    

    
class MancalaAI:
    def __init__(self, depth, state):
        self.depth = depth
        self.state = state
        

    # minimax will return the best move that the AI player can make
    # assuming the other player makes the best move they possibly can
    def minimax(self, state, depth, maximizing_player, alpha_beta):
        # base case / stopping condition
        if depth == 0 or state.terminal_state() == True:
            # print("STOPPING CONDITION")
            # print("Hit stopping condition, returning:", evaluate_state(state))
            return evaluate_state(state)
        
    
        if state.current_player == 1: # maximising player
            max_move_value = float("-inf")  # Initialize to negative infinity
            new_move_value = 0
            beta = alpha_beta
            
            
            
            # generate all possible states for the maximizing player
            for i in possible_valid_moves(state):
                if beta < max_move_value:
                    return max_move_value
                
                # print("We are maximizing this player(",state.current_player,"), here is the current board")
                new_state = state.copy()
                new_state.play(i)
                # print("Here is the board after the most recent move")
                # new_state.display_board()
                # print("now calling minimax on this board which is set to test player:", new_state.current_player)
                
                new_move_value = self.minimax(new_state, depth-1, False, max_move_value) # minimizing players' move
                
                if new_move_value > max_move_value:
                    max_move_value = new_move_value
            # print("The max value we found was", max_move_value)
            return max_move_value
                
                  
                
        else: # minimizing player
            min_move_value = float("inf")  # Initialize to positive infinity
            new_move_value = 0
            alpha = alpha_beta # Will interact with beta (min_move_value)
            
            # generate all possible states for the minimizing player
            for i in possible_valid_moves(state):
                if alpha > min_move_value:
                    return min_move_value
                
                # print("We are minimzing this player(",state.current_player,"), here is the current board")
                new_state = state.copy()
                new_state.play(i)
                # print("Here is the board after the most recent move")
                # new_state.display_board()
                # print("now calling minimax on this board which is set to test player:", new_state.current_player)
                new_move_value = self.minimax(new_state, depth-1, True, min_move_value) # maximizing players' move
                
                # print("finding min: new_move_value is", new_move_value)
                # print("finding min: min_move_value is", min_move_value)
                if new_move_value < min_move_value:
                    # print("clearly new_move_value < min_move_value")
                    min_move_value = new_move_value
            # print("The min value we found was", min_move_value)
            return min_move_value
        
        
        

    def best_move(self, state):
        
        best_move = 1
        max_move_value = float("-inf")  # Initialize to negative infinity
        new_move_value = 0
        min_move_value = float("inf")  # Initialize to negative infinity
        
        for i in possible_valid_moves(state):
            
            
            
            # print("We are maximizing this player(",state.current_player,"), here is the current board")
            new_state = state.copy()
            new_state.play(i)
            # print("Here is the board after the most recent move")
            # new_state.display_board()
            # print("now calling minimax on this board which is set to test player:", new_state.current_player)
            
            if state.current_player == 1:
                new_move_value = self.minimax(new_state, self.depth-1, False, max_move_value) # minimizing players' move
            
                # print("AI player testing if it made move of:", i,"which has value:",new_move_value)
                if new_move_value > max_move_value:
                    best_move = i
                    max_move_value = new_move_value
            else:
                new_move_value = self.minimax(new_state, self.depth-1, True, min_move_value) # minimizing players' move
            
                # print("AI player testing if it made move of:", i,"which has value:",new_move_value)
                if new_move_value < min_move_value:
                    best_move = i
                    min_move_value = new_move_value
            
        # print("AI player makes best move of:", best_move)
        return best_move

In [6]:
# this will run a certain number of games so we can test an
# AI player which uses the MiniMax algorithm against a raondomly moving player
def AI_vs_random_match_analysis(games = 100):

        p1_count = 0
        p2_count = 0
        ties_count = 0
        move_count = 0

        player_turn = 1


        # play 100 games
        for i in range(games):
            
            

            game = Mancala()
            # game.play(1)
            AIPlayer = MancalaAI(5, game)
            while game.terminal_state() == False:
                if game.current_player == 1:
                    # print("The game 'board' says its this players turn:", game.current_player)
                    # print("Random player taking their turn")
                    
                    
                    game.play(AIPlayer.best_move(game))
                else:
                    # print("The game 'board' says its this players turn:", game.current_player)
                    # print("AI player is taking their turn, here is the current board")
                    # game.display_board()
                    game.play(game.random_move_generator())
                    # game.play(AIPlayer.best_move(game))
                move_count += 1
                if game.terminal_state() == True:
                    if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
                        p1_count += 1
                    elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
                        p2_count +=1
                    else:
                        ties_count += 1
                    
        print("Player 1 (AI) won:", p1_count/games, "% of the time, and Player 2 (random) won:", p2_count/games,"of the time, in an average of:", move_count/games,"moves, ties count", ties_count)

        pass





AI_vs_random_match_analysis(100)

Player 1 (AI) won: 0.97 % of the time, and Player 2 (random) won: 0.01 of the time, in an average of: 74.7 moves, ties count 2


### Minimax questions
AI vs random, on 3 plies AI is winning about 85% of the time. On average it takes 73 moves to win.

### Minimax question: 5 plies (without alpha beta)
AI player beats a random player about 93% of the time, with minimax running on 5 plies. If I increase the plies the AI player begins to perform even better. On average it takes 75ish moves to win. My AI player is signifigantly better than random chance. It is right much more than random chance. It also is looking ahead to future potnetial game states and using that to influence its current move which is much better than random chance.

### Questions on alpha-beta
It takes about 10 seconds to run which seems super fast.
AI player wins the same as before. Same average number of moves to win.
No my results are the same as before.













### How it works
NOTE: I am using these libraries: copy, math, time, random
The AI player works on a minimax algorithem with a version of alpha-beta pruning to improve runtime. Lets look through each part of the algorithem step by step to understand how it all works. 

Before minimax is called, best_move funciton is called. Before that the AI_vs_random_match_analysis funciton is called. I'll start by explaining the outermost fucntion and then we will work our way into the more complex funcitons.

#### AI_vs_random_match_analysis function
This function runs an entered number of "games" (defualt 100) testing an AI player vs a random player. It is essentially a for loop which playes a "games" count of games, and counts up the number of times each player wins, and whenever there is a tie. Every time the AI player makes a move, the best_move function is called.

#### best_move function
This function takes in a gamestate. It then uses the possible_valid_moves function to see what moves are available. On each move, it runs the minimax funciton to get a value of how useful that move is. This means: assuming the oponenet makes the best moves they can, and AI makes the best moves it can, what would be the score at the end of the game, given the AI makes one of the possible moves available. Whichever move causes the best predicted outcome for the AI player the AI will choose as the move it makes. Note that if the AI is player 2 instead of player 1, it will choose the move with the smallest value. This is becuase minimax returns the score in terms of player 1 (player 1 score - player 2 score), so if we are player 2 we want to choose the value which gives player 1 the smallest score at the end of the game.

#### minimax function
The minimax funciton is a recursive function which alternates between maximizing player 1, and minimazing player 2. This function takes in a game state. It looks at the possible moves available  for either player 1 or 2, and then calls minimax function on each of these moves. Each call of minimax it alternates between players 1 and 2. This causes it to explore every possible move for either player 1 or 2 at each level.

Eventually it hits a stopping condition. This is when "depth" <= 0 (depth is a value we enter in ahead of time so it doen't go through every mancala game which can ever exist (that value may approach infinity)). OR when the simulated game hits a winning state. When it hits one of these stopping conditons it returns player 1 score - player 2 score, this value is how much player 1 will be winning by at the end of the game. What it finally returns to the best_move function is a value of what the AI would be expected to be winning by if it makes a particular move.

As the recursive calls dead end it returns back up through the calls. Each level, either min or max, is trying to minimize or maximize the value returned. Each max level is simulating an ideal move from player 1, where player 1 tries to maximize its score at the end of the game. Each min level is opposite.

There is also alpha-beta pruning which just makes the algorithm run faster. Here's how it works: If we are minimizing a level, and we have a specific min value, say 4. Then when we call minimax to simulate the next game state, we are maximizing. On this maximizing level we can decide this level is not nesisary to calculate if it has a value to return which is greater than 4. This is becuase when we recurse backwards and we are now minimizing, the value greater than 4 will be discarded. Result: using alpha beta pruning we can prune sections of our recursive function and prevent excess redundant calculation.


#### What I learned
i learned about minimax, how the function works internally, how to implement it with a recursive function. I also learned about alphabeta pruning and how much faster minimax can be with alpha beta pruning implemented. 
